In [5]:
import pandas as pd

In [6]:
# import pre-race file with club names
clubs=pd.read_csv('../data/rotg-clubs.csv')
print clubs.shape
clubs['Name']=clubs['first']+" "+clubs['last']
clubs.head()

(318, 12)


,first,last,gender,city,state,age,usatfnum,hmrrc,usatf,clubname,uptodate,paid,Name
0,Raymond W.,"Lee, Jr.",M,Halfmoon,NY,74,1400029425,Yes,Yes,Adirondack Athletic Club,yes,yes,"Raymond W. Lee, Jr."
1,Thomas,Dalton,M,Schenectady,NY,57,1400029581,No,Yes,Adirondack Athletic Club,yes,yes,Thomas Dalton
2,rob,picotte,M,Malta,NY,62,1400510465,Yes,Yes,Adirondack Athletic Club,yes,yes,rob picotte
3,Chris,Buckley,M,Burnt Hills,NY,54,1400713184,Yes,Yes,Adirondack Athletic Club,yes,yes,Chris Buckley
4,Christopher,Rush,M,Schenectady,NY,79,1401406259,Yes,Yes,Adirondack Athletic Club,NaN,yes,Christopher Rush


In [7]:
# import the results
results=pd.read_fwf('../data/rotg2016.txt')

results.head()

,Place,Name,No.,Age,S,City,St,Time,Pace
0,1,Eric Macknight,383,27,M,Ballston Lake,NY,19:57,5:00
1,2,Aaron Lozier,378,27,M,Albany,NY,20:22,5:06
2,3,Alex Benway,217,25,M,Saratoga Spring,NY,20:52,5:13
3,4,Shaun Donegan,278,30,M,Malta,NY,20:58,5:15
4,5,Macky Lloyd,374,25,M,Albany,NY,21:13,5:19


In [8]:
# in this step we use diflib to map 'results.Name' to the closest match in 'clubs.Name' this will ensure
# we have the same name in each
import difflib 
results.Name = results.Name.map(lambda x: difflib.get_close_matches(x, clubs.Name, cutoff=0.4)[0])

In [9]:
results=pd.merge(clubs, results, on='Name')

In [10]:
import numpy as np
import datetime


results['time'] = results.Time.apply(lambda x: datetime.datetime.strptime(x,'%M:%S'))
results['timedelta'] = results.time - datetime.datetime.strptime('00:00,0','%M:%S,%f')
results['secs'] = results['timedelta'].apply(lambda x: x / np.timedelta64(1, 's'))

results.shape

(305, 24)

In [11]:
hmrrc=results[results.hmrrc=='Yes']
hmrrc=hmrrc[['first','last','gender','age','timedelta']]
hmrrc.head()

,first,last,gender,age,timedelta
0,Raymond W.,"Lee, Jr.",M,74,00:51:32
1,rob,picotte,M,62,00:28:16
2,Chris,Buckley,M,54,00:26:11
3,Christopher,Rush,M,79,00:39:33
4,Tom,McGuire,M,64,00:29:09


In [16]:
def prixReport(agemin, agemax, gender):
    racers=hmrrc[hmrrc.gender==gender]
    racers=racers[racers.age>=agemin]
    racers=racers[racers.age<=agemax]
    
    racers=racers.sort('timedelta').iloc[0:10]
    
    return racers
    

In [17]:
for age in [[20,29],[30,39],[40,49],[50,59],[60,69],[70,99]]: 
    for gender in ['M','F']:
        print gender, age
        
        prixReport(age[0],age[1],gender)

M [20, 29]
F [20, 29]
M [30, 39]
F [30, 39]
M [40, 49]
F [40, 49]
M [50, 59]
F [50, 59]
M [60, 69]
F [60, 69]
M [70, 99]
F [70, 99]


In [19]:
f = open('../output/hmrrcp.html','w')

message = """<html>
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="chrome=1">
    <title>Runnin&#39; of the Green HMRRC Grand Prix</title>

    <link rel="stylesheet" href="stylesheets/styles.css">
    <link rel="stylesheet" href="stylesheets/github-light.css">
    <meta name="viewport" content="width=device-width">
    <!--[if lt IE 9]>
    <script src="//html5shiv.googlecode.com/svn/trunk/html5.js"></script>
    <![endif]-->
  </head>
<body><h1>Runnin' of the Green HMRRC Grand Prix Preliminary Results</h1>"""

for age in [[20,29],[30,39],[40,49],[50,59],[60,69],[70,99]]: 
    for gender in ['M','F']:
        
        table=prixReport(age[0],age[1],gender)
        
        table=table.to_html(index=False) if table.shape[0]>0 else None

        message=message+"<h2>"+gender+str(age)+"</h2>"
        
        if (table!=None):
            message=message+table
            message=message+"<br>"

        message=message+"""</body>
        </html>"""

f.write(message)
f.close()

#HTML("../output/hmrrcp.html").write_pdf("../output/rotg2016_hmrrcgrandprix.pdf")

In [76]:
def gp_table(gender):
    final=pd.DataFrame(index=range(0,10))
    
    agestrings=['open','30-39','40-49','50-59','60-69','70+']
    i=0
    
    for age in [[20,29],[30,39],[40,49],[50,59],[60,69],[70,99]]:
        table=prixReport(age[0],age[1],gender)
        table=table.reset_index(drop=True)
        
        final[str(gender)+agestrings[i]+' points']=[12,10,8,7,6,5,4,3,2,1]
        final[str(gender)+agestrings[i]]=table['first']+' '+table['last']
        final[str(gender)+agestrings[i]]=final[str(gender)+agestrings[i]].fillna("")
        i=i+1
        
    return final
        
    

In [77]:
males=gp_table('M')
females=gp_table('F')

In [78]:
from pandas import ExcelWriter
import os
def dfs2xlsx(list_dfs,xls_path = None):
    #save_xls([df1,df2],'output1.xlsx')
    if xls_path == None :
        xls_path = '~tmp.xlsx'
    writer = ExcelWriter(xls_path)
    i=0
    for n, df in enumerate(list_dfs):
        df.to_excel(writer,'Sheet1',startcol=0,startrow =i,index=False)
        i+= df.shape[0] + 2
    writer.save()
    os.system('start excel.exe %s' %(writer.path ))

In [79]:
dfs2xlsx([males,females],'../output/rotg_gp.xlsx')

In [62]:
females.size

120

In [63]:
females.shape

(10, 12)